In [3]:
import torch
import torch.nn as nn
import re
import numpy as np
import json
import os
from torch.utils.data import Dataset, DataLoader,TensorDataset
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
# from google.colab import drive
# drive.mount('/content/drive')



In [4]:
def split_get_label_from_filename(filename):
    return label_mapping.get(filename[8:-5],-1)

def get_label_from_filename(filename):
    match = re.search(r'_(\d+_?\d*)\.json$', filename)
    print(match)
    if match:
        label_key = match.group(1)
        return label_mapping.get(label_key, -1)  # 如果标签不匹配返回 -1

# 获取文件夹中的文件及其标签
def load_files_and_labels(folder_path,split=False):
    file_list = []
    labels = []
    for file in os.listdir(folder_path):
        # print("file: ", file)
        if file.endswith(".json"):
            if split:
                label = split_get_label_from_filename(file)
            else: label = get_label_from_filename(file)
            if label != -1:
                file_list.append(os.path.join(folder_path, file))
                labels.append(label)
            #print(f"File: {file}, Label: {label}")

    return file_list, labels


In [5]:
import json
import numpy as np

# 1. 提取字符并编码为整数索引
def hex_to_sequence(hex_feature):
    """
    将十六进制字符串转换为整数索引序列。
    去掉冒号并转换为字符对应的索引。
    """
    hex_chars = hex_feature.replace(":", "")
    char_to_index = {char: idx for idx, char in enumerate("0123456789abcdef")}
    return [char_to_index[char] for char in hex_chars]

def process_json(file_path):
    """
    处理 JSON 文件，提取时间戳、包长度和 raw data 特征，并对特征进行归一化。
    """
    features = []
    try:
        # 打开 JSON 文件
        with open(file_path, 'r', encoding='utf-8') as file:
            datas = json.load(file)

        # 遍历 JSON 数据，提取时间戳和包长度
        initial_timestamp = None
        for data in datas:
            try:
                timestamp = float(data["_source"]["layers"]["frame"]["frame.time_relative"])  # 时间戳
                packet_length = int(data["_source"]["layers"]["frame"]["frame.len"])  # 包长度

                # 获取数据部分，若不存在则为0
                if 'data' in data["_source"]["layers"]:
                    rawdata = data["_source"]["layers"]["data"]["data.data"]
                else:
                    rawdata = '0'

                # 将原始数据转换为整数序列
                data_feature = hex_to_sequence(rawdata)

                # 将数据填充或截断为指定长度 2832
                if len(data_feature) < 2832:  # 如果长度小于 2832，填充 0
                    data_feature += [0] * (2832 - len(data_feature))
                elif len(data_feature) > 2832:  # 如果长度大于 2832，截断
                    data_feature = data_feature[:2832]

                # 如果数据长度不符合预期，打印出来
                if len(data_feature) != 2832:
                    print(f"Data feature length mismatch: {len(data_feature)}")

                # 初始化时间戳
                if initial_timestamp is None:
                    initial_timestamp = timestamp  # 设置初始时间戳

                # 计算相对时间戳
                relative_timestamp = timestamp - initial_timestamp
                # print(type(relative_timestamp))


                timestamp_array = np.array([relative_timestamp], dtype=float)
                packet_length_array = np.array([packet_length], dtype=float)/1512
                data_feature = np.array(data_feature, dtype=float)/15

                # 将特征按顺序组合为 [时间戳, 包长度, 数据特征]
                feature = np.hstack((timestamp_array, packet_length_array,data_feature))

                # 添加到特征列表
                features.append(feature)

            except (KeyError, ValueError) as e:
                # 跳过有问题的数据包
                print(f"Skipping packet due to error: {e}")
                continue

        features_array=np.array(features)
        max_timestamp = np.max(features_array[:, 0])  # 获取最大时间戳
        # print("max")
        features_array[:, 0] = [feature[0] / max_timestamp for feature in features_array]  # 时间戳归一化
        # print("11")

    except Exception as e:
        print(Exception)
        print(f"Error processing file {file_path}: {e}")
        features_array = np.zeros((1, 2834))  # 返回空特征以避免程序中断

    return features_array

In [6]:
import nest_asyncio
nest_asyncio.apply()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义文件夹路径 /content/drive/MyDrive/202_project/202_packet_json

split = 2 # 0:100, 1:300, 2:800
# folder_path = "packet_json_split"
if split==1:
    folder_path = "packet_json_split"
elif split==2:
    folder_path = "202_packet_json_new_800"
else:
    folder_path = "202_packet_json_new"

# 定义标签映射
label_mapping = {
    "0": 0,             # static
    "0_2": 1,           # slightly_move
    "1": 2,             # move
    "4": 3              # intensely_move
}

# 调用函数获取文件和标签
data_paths, labels = load_files_and_labels(folder_path,split)
# 用于存储每个样本的特征

print(type(labels))
labels_array=np.array(labels)#[:50]
print(labels_array.shape)
print(labels_array)
labels_tensor=torch.tensor(labels_array,dtype=torch.long)

Using device: cuda
<class 'list'>
(800,)
[1 0 2 2 1 3 1 3 1 1 2 1 0 0 3 0 0 0 2 0 2 3 2 3 0 3 0 3 3 0 0 1 2 1 3 1 2
 0 2 1 3 3 0 0 2 2 2 3 0 3 3 1 2 1 1 3 0 0 3 3 2 0 1 1 1 0 0 3 0 1 3 2 3 3
 1 1 0 2 0 2 3 1 3 0 1 0 0 3 1 3 3 0 1 3 1 1 2 0 3 3 1 1 0 0 1 0 3 3 2 0 2
 2 2 3 0 3 2 3 0 3 1 0 0 3 2 0 0 0 0 2 0 3 0 2 3 3 1 0 3 0 1 2 1 2 0 2 0 3
 0 1 1 0 2 1 3 0 0 0 3 2 1 1 2 3 3 0 1 3 1 3 3 3 2 1 1 2 3 2 1 1 1 2 2 0 0
 0 1 2 2 2 2 0 1 2 1 1 1 2 3 0 1 0 2 3 2 2 0 2 1 1 3 0 3 2 2 1 2 3 0 2 3 0
 1 1 0 3 2 1 3 1 2 3 2 1 3 1 0 0 1 3 1 1 2 0 3 3 1 3 3 0 3 3 3 0 2 2 1 1 0
 3 2 2 3 3 2 1 1 2 2 1 3 3 3 0 0 3 0 2 3 0 2 0 2 0 2 2 0 0 0 2 2 1 2 0 0 2
 3 3 3 1 0 2 2 0 2 1 3 0 0 1 3 0 3 1 0 0 1 0 2 2 2 2 3 2 3 3 2 1 2 1 3 3 1
 1 3 1 1 2 1 1 1 3 0 3 0 3 0 0 2 2 0 3 1 1 0 1 1 0 3 0 0 3 3 0 1 0 1 0 3 3
 0 0 1 1 0 0 2 0 3 1 3 3 3 0 0 2 2 2 2 1 2 2 0 1 0 2 1 1 3 0 2 2 2 2 3 1 1
 1 2 3 1 1 2 1 2 3 0 3 3 3 1 3 2 2 1 1 1 2 2 1 2 2 0 0 0 3 2 3 0 0 2 1 3 3
 3 2 3 0 3 3 1 2 2 0 0 0 0 0 3 2 1 0 0 0 2 0 3 2 1 0 1 3 2 

In [7]:
features_list = []
for data_path in data_paths:
  # count +=1
  #print(data_path)
  features = process_json(data_path)
  features_list.append(features)
  #print(features.shape)
print(f"生成了 {len(features_list)} 个样本特征")
print(f"第一个样本的特征形状: {features_list[0].shape}")

生成了 800 个样本特征
第一个样本的特征形状: (1486, 2834)


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load features and labels (as previously defined)
features_list = []
for data_path in data_paths:
    features = process_json(data_path)
    features_list.append(features)

# Flatten features and ensure all have the same shape
if split == 1:
    fixed_time_steps = 3000
elif split == 2:
    fixed_time_steps = 1500
else:
    fixed_time_steps = 12000


target_dim = 512  # 降到 256 维
pca = PCA(n_components=target_dim)

# 对每个特征矩阵进行降维
reduced_features_list = []
for feature in features_list:
    reduced_feature = pca.fit_transform(feature)  # 对特征维度进行降维
    reduced_features_list.append(reduced_feature)

# 截断或补零到固定时间步长

aligned_features = []
for feature in reduced_features_list:
    if feature.shape[0] > fixed_time_steps:
        truncated = feature[:fixed_time_steps, :]
    else:
        truncated = np.pad(feature, ((0, fixed_time_steps - feature.shape[0]), (0, 0)), mode='constant')
    aligned_features.append(truncated)

# Flatten time-step dimension into a single feature vector
aligned_features = [feature.flatten() for feature in aligned_features]
features_array = np.array(aligned_features)

# Convert labels to NumPy array
labels_array = np.array(labels)

# Split the dataset into train and validation sets
if split == 1:
    train_features = features_array[:210]
    train_labels = labels_array[:210]
    eval_features = features_array[210:]
    eval_labels = labels_array[210:]
elif split == 2:
    train_features = features_array[:600]
    train_labels = labels_array[:600]
    eval_features = features_array[600:]
    eval_labels = labels_array[600:]
else:
    train_features = features_array[:70]
    train_labels = labels_array[:70]
    eval_features = features_array[70:]
    eval_labels = labels_array[70:]

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=20)  # You can adjust the number of neighbors

# Train the KNN classifier
knn.fit(train_features, train_labels)

# Evaluate on the validation set
eval_predictions = knn.predict(eval_features)
eval_accuracy = accuracy_score(eval_labels, eval_predictions)

print(f"Validation Accuracy: {eval_accuracy * 100:.2f}%")

# Evaluate on the training set (optional)
train_predictions = knn.predict(train_features)
train_accuracy = accuracy_score(train_labels, train_predictions)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Example usage to predict new samples:
# new_sample = aligned_features_flat[0]  # Example feature
# prediction = knn.predict([new_sample])
# print(f"Predicted class: {prediction[0]}")

Validation Accuracy: 24.50%
Training Accuracy: 25.50%
